In [1]:
import os
import requests
import subprocess
import geopandas as gpd
import pandas as pd

In [2]:
today = pd.Timestamp.today().strftime('%Y-%m-%d')

In [3]:
# Requires local https://github.com/openaddresses/pyesridump install

In [4]:
# Define the URL and output file path
layer_url = "https://services2.arcgis.com/zNjnZafDYCAJAbN0/ArcGIS/rest/services/Street_ROW_Trees/FeatureServer/0"
output_file = "../data/raw/cities/pasadena.geojson"

In [5]:
# Use subprocess to call esri2geojson
subprocess.run(["esri2geojson", layer_url, output_file])

2024-05-28 12:02:39,224 - cli.esridump - INFO - Built 58 requests using resultOffset method
2024-05-28 12:02:53,621 - cli.esridump - INFO - pause for 10 seconds
2024-05-28 12:03:35,621 - cli.esridump - INFO - pause for 10 seconds
2024-05-28 12:03:46,390 - cli.esridump - INFO - retry pause 0
2024-05-28 12:04:06,515 - cli.esridump - INFO - retry pause 1
2024-05-28 12:04:18,025 - cli.esridump - INFO - retry pause 0
2024-05-28 12:04:28,379 - cli.esridump - INFO - pause for 10 seconds
2024-05-28 12:04:39,432 - cli.esridump - INFO - retry pause 0
2024-05-28 12:04:59,943 - cli.esridump - INFO - retry pause 1
2024-05-28 12:05:10,538 - cli.esridump - INFO - retry pause 0
2024-05-28 12:05:20,871 - cli.esridump - INFO - pause for 10 seconds
2024-05-28 12:05:33,449 - cli.esridump - INFO - pause for 10 seconds
2024-05-28 12:05:43,763 - cli.esridump - INFO - retry pause 0
2024-05-28 12:05:53,766 - cli.esridump - INFO - pause for 10 seconds
2024-05-28 12:06:06,358 - cli.esridump - INFO - pause for 10

CompletedProcess(args=['esri2geojson', 'https://services2.arcgis.com/zNjnZafDYCAJAbN0/ArcGIS/rest/services/Street_ROW_Trees/FeatureServer/0', '../data/raw/cities/pasadena.geojson'], returncode=0)

In [6]:
# Read the GeoJSON file into a GeoDataFrame
street_trees_gdf = gpd.read_file('../data/raw/cities/pasadena.geojson')

In [7]:
# Display the dataframe
street_trees_gdf.head()

,OBJECTID,Tree_Rec,Alt_Tree_ID,Common_Name,Genus,Species,Status_Text,Address,Street_Direction,Street_Name,Street_Type,Street_Suffix,Trunk_Dia,X_Coordinate,Y_Coordinate,Classification_Text,geometry
0,2,69861,13,CHINESE ELM,ULMUS,PARVIFOLIA,Active,850.0,,ADELAIDE,DR,,16.0,6.528584e+06,1.880864e+06,Street Tree,POINT (-118.10936 34.16065)
1,3,69862,15,SAWTOOTH ZELKOVA,ZELKOVA,SERRATA,Active,858.0,None,ADELAIDE,DR,None,16.0,6.528584e+06,1.880894e+06,Street Tree,POINT (-118.10936 34.16073)
2,4,69863,16,CHINESE ELM,ULMUS,PARVIFOLIA,Active,859.0,None,ADELAIDE,DR,None,14.0,6.528544e+06,1.880891e+06,Street Tree,POINT (-118.10949 34.16072)
3,5,69864,17,CHINESE ELM,ULMUS,PARVIFOLIA,Active,867.0,None,ADELAIDE,DR,None,24.0,6.528545e+06,1.880926e+06,Street Tree,POINT (-118.10949 34.16082)
4,6,69865,18,CHINESE ELM,ULMUS,PARVIFOLIA,Active,867.0,None,ADELAIDE,DR,None,21.0,6.528544e+06,1.880963e+06,Street Tree,POINT (-118.10949 34.16092)


In [8]:
len(street_trees_gdf)

57070

In [9]:
street_trees_gdf['fetched_on'] = today
street_trees_gdf['municipality'] = 'Pasadena'
street_trees_gdf['source'] = 'Esri endpoint'

In [10]:
street_trees_gdf.columns = street_trees_gdf.columns.str.lower()

In [11]:
street_trees_gdf.columns

Index(['objectid', 'tree_rec', 'alt_tree_id', 'common_name', 'genus',
       'species', 'status_text', 'address', 'street_direction', 'street_name',
       'street_type', 'street_suffix', 'trunk_dia', 'x_coordinate',
       'y_coordinate', 'classification_text', 'geometry', 'fetched_on',
       'municipality', 'source'],
      dtype='object')

In [12]:
street_trees_gdf['lon'] = street_trees_gdf.geometry.x
street_trees_gdf['lat'] = street_trees_gdf.geometry.y

In [13]:
street_trees_gdf['address'] = street_trees_gdf['address'].astype(str).str.replace('.0', '')

In [14]:
street_trees_gdf[['tree_rec', 'common_name', 'genus',
       'species', 'status_text', 'address', 'street_direction', 'street_name',
       'street_type', 'trunk_dia', 'fetched_on',
       'municipality', 'lon', 'lat']].to_csv('../data/raw/cities/pasadena.csv', index=False)